In [ ]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
from scipy.special import i0, i1
from tqdm import tqdm 
import sys
from pathlib import Path

Path("./Paper Figures/").mkdir(parents=True, exist_ok=True)
Path("./Paper Data/").mkdir(parents=True, exist_ok=True)

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
epsilon_list = [0.1, 0.25, 0.5]
sigma_list = [0.5, 2, 4]

In [ ]:
def V_tilde_prime(x, sigma):
    if abs(x) > 2:
        return x**3 - x
    else:
        return (x**3 - x) / i0(x**2 / (2*sigma))**2 + x / i0(x**2 / (2*sigma))**3 * i1(x**2 / (2*sigma))

r = 3
for j, epsilon in enumerate(epsilon_list):
    fig, ax = plt.subplots(1, 3)
    for k, sigma in enumerate(sigma_list):
        A_new = np.load(f"./Paper Data/experiment3-ma-42-{j}-{k}.npy")

        x = np.linspace(-2, 2, 500)
        ax[k].plot(x, np.sum(A_new[:, None]*np.array([x**i for i in range(r+1)]), axis=0), label="Approximation $\widetilde b$", linewidth=3, color='C1')
        ax[k].plot(x, [V_tilde_prime(xi, sigma) for xi in x], label="Exact $b$", linewidth=3, linestyle='dashed', color='C0')
        
        ax[k].set_ylim(-5, 5)
        if k > 0:
            ax[k].set_yticklabels([])
        if j < 2:
            ax[k].set_xticklabels([])
    fig.set_size_inches(7.5, 2)
    #plt.savefig(f'./Paper Figures/experiment3-ma-42-{j}.pdf', bbox_inches='tight')
    plt.show()